# Vertex AI Agent SDK Example - Company News Agent Application

## Overview

Vertex AI Agent SDK is a platform for creating and managing GenAI applications with Agents, that can use function and extension based tools to connect large language models to external systems via APIs.

These external systems can provide LLMs with real-time data and perform data processing actions on their behalf. You can use pre-built (e.g. code-interpreter) or your own extensions in Vertex AI Agents SDK.

<!-- Learn more about [Vertex AI Extensions](https://cloud.google.com/vertex-ai/generative-ai/docs/extensions/overview).-->


### Objective

In this example, you learn how to create an extension service backend on Cloud Run, register the extension with Vertex AI Agent SDK, and then use the Agent Application in a user session.

More details can be found in the project's [GitHub repository](https://github.com/neo4j-examples/neo4j-gcp-vertex-ai-langchain/tree/main/companies-extension).

You can answer of complex questions that combine LLM language skills with the data from the extension, allowing for multiple chained retrievals driven by the LLM.

The steps performed include:

- The Neo4j Company News database
- Creating the extension service running on Cloud Run
- Creating an OpenAPI 3.1 YAML file for the Cloud Run service
- Creating an Vertex AI Agent SDK application
- Registering the service as an extension for an agent with Vertex AI Agent SDK
- Using a session to respond to user complex user queries which will use a number of agent calls behind the scenes


## How it works

This notebook provides an external Neo4j Extension to be used with the Agent of our application.

The extension makes a number of REST endpoints available for the agent to query a company news graph with articles about companies their industries and executives involved with the companies.

It is deployed as a Flask app in a Docker container with Google Cloud run.

This guide assumes that you are somewhat familiar with

* [Vertex AI Agent SDK](https://cloud.google.com/vertex-ai/generative-ai/docs/agent-api/overview)
* [LangChain](https://python.langchain.com/docs/get_started/introduction)
* [OpenAPI specification](https://swagger.io/specification/)
* [Cloud Run](https://cloud.google.com/run/docs)

**_NOTE_**:

Your account needs to have permissions and API enabled for Vertex AI and Cloud Run.

The notebook has been tested in the following environment: Python version = 3.11

## Before you begin

### Set up your Google Cloud project

**The following steps are required, regardless of your notebook environment.**

1. [Select or create a Google Cloud project](https://console.cloud.google.com/cloud-resource-manager). When you first create an account, you get a $300 free credit towards your compute/storage costs.
1. [Make sure that billing is enabled for your project](https://cloud.google.com/billing/docs/how-to/modify-project).
1. [Enable the Vertex AI API](https://console.cloud.google.com/flows/enableapi?apiid=aiplatform.googleapis.com).
1. If you are running this notebook locally, you need to install the [Cloud SDK](https://cloud.google.com/sdk).
1. Your project must also be allowlisted for the Vertex AI Extension Private Preview.
1. This notebook requires that you have the following permissions for your GCP project:
- `roles/aiplatform.user`

### Set your project ID

**If you don't know your project ID**, try the following:
* Run `gcloud config list`.
* Run `gcloud projects list`.
* See the support page: [Locate the project ID](https://support.google.com/googleapi/answer/7014113)

##Authenticate

In [1]:
from google.colab import auth
auth.authenticate_user()

##Imports and Initialization


Make sure to initialize aiplatform with your projectID, location, and api endpoint. You need to initialize aiplatform before performing any of the other imports.

In [1]:
PROJECT_ID = "vertex-ai-neo4j-extension" # @param {type:"string"}
LOCATION = "us-central1"  # @param {type: "string"}
API_ENDPOINT = 'us-central1-aiplatform.googleapis.com'

from google.cloud import aiplatform

aiplatform.init(project=PROJECT_ID, location=LOCATION, api_endpoint=API_ENDPOINT)

In [3]:
!gcloud config set project $PROJECT_ID

Updated property [core/project].


##Download most recent copy of Vertex Agents SDK

In [4]:
!gsutil cp gs://vertex_agents_private_releases/vertex_agents/google_cloud_aiplatform-1.61.dev20240814+vertex.agents-py2.py3-none-any.whl .
!pip install --quiet --upgrade --force-reinstall -q google_cloud_aiplatform-1.61.dev20240814+vertex.agents-py2.py3-none-any.whl --no-warn-conflicts
!pip install --quiet -U "pandas==2.2.2"
!pip install --quiet -U 'numpy<2'

# Restart the kernel runtime to load the private preview SDK
import IPython
app = IPython.Application.instance()
app.kernel.do_shutdown(True)

Copying gs://vertex_agents_private_releases/vertex_agents/google_cloud_aiplatform-1.61.dev20240814+vertex.agents-py2.py3-none-any.whl...
/ [1 files][  5.3 MiB/  5.3 MiB]                                                
Operation completed over 1 objects/5.3 MiB.                                      
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 60.9/60.9 kB 4.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 142.2/142.2 kB 10.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 209.0/209.0 kB 16.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 239.1/239.1 kB 17.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 130.5/130.5 kB 10.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 294.6/294.6 kB 19.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 5.8/5.8 MB 63.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 16.3/16.3 MB 56.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 229.9/229.9 kB 

{'status': 'ok', 'restart': True}

## Company News Graph

Our example datasets will be a small subset (250k entities) of [diffbot's](https://diffbot.com/) global Knowledge Graph (50bn entities).

It contains organizations, people in their leadership, locations and industries.

Additionally articles mentioning these companies, which are chunked and indexed as vector embeddings.

![](https://camo.githubusercontent.com/2ebaff2ceb74cd8af8f7ef8a5a4791ef92ea8f631c0fc41d828eb8ae9c7bf553/68747470733a2f2f692e696d6775722e636f6d2f6c574a5a5345652e706e67)

You can access and query a read only version of that graph with the following credentials:

* URL: https://demo.neo4jlabs.com:7473
* Username: companies
* Password: companies
* Database: companies

## Neo4j Company News Extension

This is the Flask application, connecting to Neo4j and hosting 6 different endpoints to retrieve information about companies, article, people, industries etc.

### Endpoints

Each endpoint takes parameters via query string and returns it's response as JSON

* `/industries` - List of Industry names
* `/companies` - List of Companies (id, name, summary) by fulltext `search`
* `/companies_in_industry` - Companies (id, name, summary) in a given industry by `industry`
* `/articles_in_month` - List of Articles (id, author, title, date, sentiment) in a month timeframe from the given `date` (yyyy-mm-dd)
* `/article` - Single Article details (id, author, title, date, sentiment, site, summary, content) by article `id`
*  `/companies_in_articles` - Companies (id, name, summary) mentioned in articles by list of article `ids`
* `/people_at_company` - People (name, role) associated with a company by company `id`

It uses a publicly hosted dataset with read-only credentials for the "companies" graph.

In [2]:
# Connection Details

NEO4J_URI='neo4j+s://demo.neo4jlabs.com'
NEO4J_USERNAME='companies'
NEO4J_PASSWORD='companies'
NEO4J_DATABASE='companies'

In [3]:
import os
if not os.path.exists("extension"):
    os.mkdir("extension")

### Flask Application

The code for the application (`extension.py`), `Dockerfile` and `requirements.txt`is also also available on [GitHub](https://github.com/neo4j-examples/neo4j-gcp-vertex-ai-langchain/tree/main/companies-extension).

In [4]:
%%writefile extension/extension.py
import os
from flask import Flask, jsonify, request
from neo4j import GraphDatabase

app = Flask(__name__)

URI = os.getenv('NEO4J_URI', 'neo4j+s://demo.neo4jlabs.com')
AUTH = (os.getenv('NEO4J_USERNAME','companies'),os.getenv('NEO4J_PASSWORD','companies'))

driver = GraphDatabase.driver(URI, auth = AUTH)

@app.route("/industries", methods=["GET"])
def industries():
    """
    List of Industry names
    """

    query = """
    MATCH (i:IndustryCategory) RETURN i.name as industry
    """
    records, _, _ = driver.execute_query(query, _database="companies")
    results = [r['industry'] for r in records]
    return jsonify({ "output": results })

@app.route("/companies", methods=["GET"])
def companies():
    """
    List of Companies (id, name, summary) by fulltext search
    """

    query = """
    CALL db.index.fulltext.queryNodes('entity', $search, {limit: 25})
    YIELD node as c, score WHERE c:Organization
    AND not exists { (c)<-[:HAS_SUBSIDARY]-() }
    RETURN c.id as id, c.name as name, c.summary as summary
    """
    args = request.args
    search = args.get("search")
    records, _, _ = driver.execute_query(query, search=search, _database="companies")
    results = [{"Company": r['name'], "Id": r['id'], "Summary":r['summary']} for r in records]
    return jsonify({ "output": results })

@app.route("/companies_in_industry", methods=["GET"])
def companies_in_industry():
    """
    Companies (id, name, summary) in a given industry by name
    """
    query = """
    MATCH (:IndustryCategory {name:$industry})<-[:HAS_CATEGORY]-(c)
    WHERE not exists { (c)<-[:HAS_SUBSIDARY]-() }
    RETURN c.id as id, c.name as name, c.summary as summary
    """
    args = request.args
    industry = args.get("industry")
    records, _, _ = driver.execute_query(query, industry=industry, _database="companies")
    results = [{"Company": r['name'], "Id": r['id'], "Summary":r['summary']} for r in records]
    return jsonify({ "output": results })

@app.route("/articles_in_month", methods=["GET"])
def articles_in_month():
    """
    List of Articles (id, author, title, date, sentiment) in a month timeframe from the given date
    """
    query = """
    match (a:Article)
    where date($date) <= date(a.date)  < date($date) + duration('P1M')
    return a.id as id, a.author as author, a.title as title, toString(a.date) as date, a.sentiment as sentiment
    limit 25
    """
    args = request.args
    date = args.get("date")

    records, _, _ = driver.execute_query(query, date=date, _database="companies")
    results = [{"Article": r['title'], "Id": r['id'], "Date": r['date'], "Sentiment":r['sentiment'], "Author":r['author']} for r in records]
    return jsonify({ "output": results })

@app.route("/article", methods=["GET"])
def article():
    """
    Single Article details (id, author, title, date, sentiment, site, summary, content) by article id
    """

    query = """
    match (a:Article)-[:HAS_CHUNK]->(c:Chunk)
    where a.id = $id
    with a, c order by id(c) asc
    with a, collect(c.text) as content
    return a.id as id, a.author as author, a.title as title, toString(a.date) as date,
    a.summary as summary, a.siteName as site, a.sentiment as sentiment, content
    """
    args = request.args
    id = args.get("id")

    records, _, _ = driver.execute_query(query, id=id, _database="companies")
    results = [{"Article": r['title'], "Id": r['id'], "Date": r['date'], "Sentiment":r['sentiment'], "Author":r['author'],
                "Site": r['site'], "Summary": r['summary'], "Content": r['content']} for r in records]
    return jsonify({ "output": results })

@app.route("/companies_in_articles", methods=["GET"])
def companies_in_articles():
    """
    Companies (id, name, summary) mentioned in articles by list of article ids
    """

    query = """
    MATCH (a:Article)-[:MENTIONS]->(c)
    WHERE a.id in $ids AND not exists { (c)<-[:HAS_SUBSIDARY]-() }
    RETURN c.id as id, c.name as name, c.summary as summary
    """
    args = request.args
    ids = args.get("ids").split(",")

    records, _, _ = driver.execute_query(query, ids=ids, _database="companies")
    results = [{"Company": r['name'], "Id": r['id'], "Summary":r['summary']} for r in records]
    return jsonify({ "output": results })


@app.route("/people_at_company", methods=["GET"])
def people_at_company():
    """
    People (id, name, summary) associated with a company by company id
    """

    query = """
    MATCH (c:Organization)-[role]-(p:Person) WHERE c.id = $id
    RETURN replace(type(role),"HAS_","") as role, p.name as name
    """
    args = request.args
    id = args.get("id")

    records, _, _ = driver.execute_query(query, id=id, _database="companies")
    results = [{"Person": r['name'], "Role": r['role']} for r in records]
    return jsonify({ "output": results })

if __name__ == "__main__":
    app.run(debug=True, host="0.0.0.0", port=int(os.environ.get("PORT", 8080)))

Writing extension/extension.py


In [5]:
%%writefile extension/requirements.txt
Flask==2.3.3
gunicorn==23.0.0
neo4j==5.22.0

Writing extension/requirements.txt


In [6]:
%%writefile extension/Dockerfile

FROM python:3.11-slim

ENV PYTHONUNBUFFERED=True

ENV APP_HOME=/app
WORKDIR $APP_HOME
COPY . ./

RUN pip install --no-cache-dir -r requirements.txt

CMD exec gunicorn --bind :$PORT --workers 1 --threads 8 --timeout 0 extension:app

Writing extension/Dockerfile


In [7]:
%%writefile extension/.dockerignore
Dockerfile
README.md
*.pyc
*.pyo
*.pyd
__pycache__
.pytest_cache

Writing extension/.dockerignore


Next, you deploy the service to Cloud Run. However, you might need to log in once more to deploy.

In [ ]:
!gcloud auth login

In [16]:
!gcloud run deploy extension --region=us-central1 --source extension --no-user-output-enabled

List the most recent Cloud Run service that was deployed, then you'll copy its URL to the next cell:

In [14]:
!gcloud run services list | sort -k 3 | head -2

   SERVICE    REGION       URL                                                 LAST DEPLOYED BY          LAST DEPLOYED AT
✔  extension  us-central1  https://extension-990868019953.us-central1.run.app  michael.hunger@neo4j.com  2024-10-02T07:39:50.100659Z


Copy paste the output from the previous command here

In [11]:
# @title Copy paste the output from the previous command here
service_url = "https://extension-990868019953.us-central1.run.app"  # @param {type:"string"}

### Test the deployed service

First, check that your service can accept simple HTTP `GET` requests:

In [18]:
import requests

url = f'{service_url}/companies_in_industry?industry=Electronic%20Products%20Manufacturers'

r = requests.get(url, headers={ 'Accept': 'application/json' })

print(f"Status Code: {r.status_code}, Content: {r.text}")


Status Code: 200, Content: {"output":[{"Company":"Altium","Id":"EJH622s4BMu2i04Ye7J6fxQ","Summary":"Software company"},{"Company":"Northrop Grumman","Id":"EXjyKMX-3MEG_BMYe08uKtg","Summary":null},{"Company":"Microsemi","Id":"EgtXOU4GjMSuesdkX0nz_Qg","Summary":"Communications corporation"},{"Company":"Nice Systems","Id":"EsLssnhntPTqbf_OUYWJ5oQ","Summary":null},{"Company":"Skyworks Solutions","Id":"EzY1DUFnTNYang87g0sX-2Q","Summary":"American semiconductor manufacturer"},{"Company":"Murrietta Circuits","Id":"Euff3XPIoPkGSqQJSwxk8wg","Summary":null},{"Company":"Airwolf 3D","Id":"E8Hvn4tyBOmGaT1zWRMKlJw","Summary":null},{"Company":"Xi'an System Sensor Electronics","Id":"EUdCZjJ7dOYOVFy9CcmAfvQ","Summary":"Manufacturing company based in Long\u2019an Xian, Guangxi Zhuang Autonomous Region, China and owned by Honeywell"},{"Company":"Bunker Ramo","Id":"EuQUgt5BHPUKqaOG8nrXBcg","Summary":"Manufacturing company owned by Honeywell founded in 1964"},{"Company":"Intermec","Id":"Eb7CrOtCJMQ-Mm5VePc

**NOTE:** If this fails with an 403 error, you need to give your service `Cloud Run Invoker` permission to an `allUsers` principal. Select the service in your [Cloud Run](https://console.cloud.google.com/run) and use the "Permissions" button.

### Create an OpenAPI spec

Your Vertex Extension requires an OpenAPI 3.1 YAML file that defines routes, URL, HTTP methods, requests, and responses from your "backend" service. The following code creates a YAML file that you can also upload to your Cloud Storage bucket.

It is important to be very specific in the descriptions of the functions, their parameters and return types. That will the LLM to make informed decision about what to extract from the user requests and history and what to pass to the extension.

We chose to use structured responses (not just text) as response types, so that the LLM can make more informed decisions, e.g. also using the key-names and types.

In [19]:
if not os.path.exists("extension-api"):
    os.mkdir("extension-api")


### OpenAPI 3.1.0 YAML Spec for the Neo4j Company Service

In [20]:
COMPANY_NEWS_YAML =f"""
openapi: "3.1.0"
info:
  version: 1.0.0
  title: neo4j_extension
  description: Service for company news information, industries, articles, people at the companies and more.
servers:
  - url: { service_url }
paths:
  /companies:
    get:
      operationId: companies
      description: List of Companies (id, name, summary) by fulltext search
      parameters:
        - name: search
          in: query
          description: Part of a name of a company to search for
          required: true
          schema:
            type: string
      responses:
        "200":
          description: Returns List of Companies (id, name, summary) as JSON
          content:
            application/json:
              schema:
                type: array
                items:
                  type: object
                  properties:
                    id:
                      type: string
                    name:
                      type: string
                    summary:
                      type: string
  /industries:
    get:
      operationId: industries
      description: List of Industry names
      responses:
        "200":
          description: Returns List of Industry Names as JSON
          content:
            application/json:
              schema:
                type: array
                items:
                  type: string
  /companies_in_industry:
    get:
      operationId: companies_in_industry
      description: List of Companies (id, name, summary) in a certain industry by name
      parameters:
        - name: industry
          in: query
          description: Exact industry name from the industries operation
          required: true
          schema:
            type: string
      responses:
        "200":
          description: Returns List of Companies (id, name, summary) as JSON
          content:
            application/json:
              schema:
                type: array
                items:
                  type: object
                  properties:
                    id:
                      type: string
                    name:
                      type: string
                    summary:
                      type: string
  /articles_in_month:
    get:
      operationId: articles_in_month
      description: List of Articles (id, author, title, date, sentiment) in a month timeframe from the given date
      parameters:
        - name: date
          in: query
          description: Start date of the month in yyyy-mm-dd format, e.g. 2021-01-01
          required: true
          schema:
            type: string
      responses:
        "200":
          description: Returns List of Articles (id, author, title, date, sentiment) as JSON
          content:
            application/json:
              schema:
                type: array
                items:
                  type: object
                  properties:
                    id:
                      type: string
                    author:
                      type: string
                    title:
                      type: string
                    date:
                      type: string
                    sentiment:
                      type: number
  /article:
    get:
      operationId: article
      description: Single Article details (id, author, title, date, sentiment, site, summary, content) by article id
      parameters:
        - name: id
          in: query
          description: Single article id
          required: true
          schema:
            type: string
      responses:
        "200":
          description: Single Article details (id, author, title, date, sentiment, site, summary, content) as JSON
          content:
            application/json:
              schema:
                type: array
                items:
                  type: object
                  properties:
                    id:
                      type: string
                    author:
                      type: string
                    title:
                      type: string
                    date:
                      type: string
                    site:
                      type: string
                    summary:
                      type: string
                    content:
                      type: string
                    sentiment:
                      type: number
  /companies_in_articles:
    get:
      operationId: companies_in_articles
      description: Companies (id, name, summary) mentioned in articles by list of article ids
      parameters:
        - name: ids
          in: query
          description: Comma separated list of article ids
          required: true
          schema:
            type: string
      responses:
        "200":
          description: Returns List of Companies (id, name, summary) as JSON
          content:
            application/json:
              schema:
                type: array
                items:
                  type: object
                  properties:
                    id:
                      type: string
                    name:
                      type: string
                    summary:
                      type: string
  /people_at_company:
    get:
      operationId: people_at_company
      description: People (name, role) associated with a company by company id
      parameters:
        - name: id
          in: query
          description: Single company id
          required: true
          schema:
            type: string
      responses:
        "200":
          description: People (name, role) associated with a company as JSON
          content:
            application/json:
              schema:
                type: array
                items:
                  type: object
                  properties:
                    name:
                      type: string
                    role:
                      type: string
"""

In [21]:
%store COMPANY_NEWS_YAML >extension-api/extension.yaml

Writing 'COMPANY_NEWS_YAML' (str) to file 'extension-api/extension.yaml'.


### Extension up and running - Next Step Agent Application

Now that our API is ready to serve requests, and we have an specification for its functions, we can create our Vertex AI Agent application and register the endpoints as extension tool to be used to retrieve information during a conversation.

#Vertex AI Agent Applications

##Create Vertex AI Agent App

There are two options for creating an app.


1.   use the `App.create` function, which will create a new app for you
2.   initialize a Vertex Agents `App` object with the resource name of a previously created App


Here we create our new **Company News** GenAI application to manage our agents and serve as entry point for our user sessions.



In [22]:
from google.cloud.aiplatform.private_preview.vertex_agents.app import App, Session
from google.cloud.aiplatform.private_preview.vertex_agents.agent import Agent
from vertexai.preview.extensions import Extension

In [30]:
app = App.create(display_name='Company News App',
                 description='An application to inform people about company news for specific industries and people working in those companies')

# To load an existing app
# app = App("projects/<project_id>/locations/<location_id>/apps/<app_id>")

##List Apps

You can list all the apps in a project, which will return a list of operational Apps.

Alternatively, you can get a user-friendly mapping of App display names and resource names.

In [ ]:
all_apps = App.list_apps()

for a in all_apps:
  print(a.display_name, a.description, a.app_name)

# Agent Tools

Our application can be augmented with two types of tools:

1. *Function call tools*, that provide a set of function signatures and can be selected and provided with parameters by the LLM but we need to take care of the execution ourselves, and provide the responses back to the session.

2. *Extension tools* that use already deployed API endpoints like the one we just created. Those are automatically called by the Agent as needed and responses integrated in multi-step reasoning processes and the session history/memory.

You can read more in the docs about [Function Call Tools](https://cloud.google.com/vertex-ai/generative-ai/docs/agent-api/create-agent#function_tool).

But here we're using the [Extension tools](https://cloud.google.com/vertex-ai/generative-ai/docs/agent-api/create-agent#extension_tool) as shown below.

###Extension Tools

You can integrate both first party and third party (like our) Extension tools into your agent.

To get a reference to a first party (Google) extension, you can use the `Extensions.from_hub(...)` method, for instance a code-interpreter: `Extension.from_hub("code_interpreter")`

More details can be found in https://cloud.google.com/vertex-ai/generative-ai/docs/extensions/create-extension.


###Custom Extension Tools

To create a custom extension, you will need to specify the display name, description, and manifest of your extension.

You can specify the Open API spec in YAML string format, and set the `open_api_yaml`, or you can upload the YAML file to a Google Cloud Storage bucket, and reference it in `open_api_gcs_uri`.

We have already done that, and the specifiction is stored in the `COMPANY_NEWS_YAML` variable.

### Extension Tooling

The `Extensions` have a few utility methods, like listing and deleting running extensions, as they are additive, i.e. you have to clean up previously registered extensions.

In [ ]:
Extension.list()

In [ ]:
for e in Extension.list():
  e.delete()

### Create Extension

Now we can create our extension with a

* display name
* description
* manifest with name, description,
 * openapi-spec YAML
 * auth config (we don't need auth for our read-only datasource)

 For more information on Authentication methods for an Extension see the [documentation](https://cloud.google.com/vertex-ai/generative-ai/docs/extensions/create-extension#specify-authentication).

In [27]:
company_news_extension = Extension.create(
    display_name = "Company News",
    description = "Service for company news information, industries, articles, people at the companies and more.",
    manifest = {
        "name": "company_news",
        "description": "Company News Extension",
        "api_spec": {
            "open_api_yaml": COMPANY_NEWS_YAML,
        },
        "auth_config": {
            "auth_type": "NO_AUTH",
        },
    },
)

INFO:vertexai.extensions._extensions:Creating Extension
INFO:vertexai.extensions._extensions:Create Extension backing LRO: projects/990868019953/locations/us-central1/extensions/3661655195470790656/operations/2420060998661570560
INFO:vertexai.extensions._extensions:Extension created. Resource name: projects/990868019953/locations/us-central1/extensions/3661655195470790656
INFO:vertexai.extensions._extensions:To use this Extension in another session:
INFO:vertexai.extensions._extensions:extension = vertexai.preview.extensions.Extension('projects/990868019953/locations/us-central1/extensions/3661655195470790656')


## Agents

Agents exist under an App. Agents can perform small, specific tasks utilizing Vertex Extensions and function calling. Agents require a display name, and a set of instructions.

When creating an agent, you must specify the following:


1.   Display Name
2.   Instructions - the detailed instructions the Agent should follow. This is where you instruct your agent when to call your functions / extensions and with what parameters.

You also have the option to specify the extensions and functions you will be using. Please note that if you reference an extension or function in the instructions, you ***must*** include an extension/function with the same display name in the extensions/functions lists.

##Create Agent
You can create an agent for your app by using the add_agent method. When creating an agent, you must specify all the required fields discussed above.



In [31]:
DISPLAY_NAME = "Company News Agent"
INSTRUCTIONS = "You are an expert in company information and news. You have access to sources like articles, companies and the people working there to answer user questions."

created_agent = app.add_agent(display_name=DISPLAY_NAME,
                              instructions=INSTRUCTIONS)

### Create a single Agent App

If you want to create an App and Agent in a single turn (i.e. avoiding using create_app -> add_agent) you can use `App.create_single_agent_app`. You must specify the agent display name, and then optionally specify tools for the agent, and the App's display name and description. This function returns an operable App.

##Update Agent

You can update any and all of the fields of an Agent by specifying them to the Agent.update function. Any fields that aren't specified will not be updated.


### Managing an applications Agents

Currently only *one* agent is possible per application, so you would have to combine your extensions into one Agent and add different instructions for different extensions.

The application class offers some methods for listing and managing agents, which are shown below. If you had already an agent registered for you app you need to remove it before adding a new one.

In [33]:
app.list_agents()

[
 agent_name: projects/990868019953/locations/us-central1/apps/992199292905062400/agents/5448665605571870720
 display_name: Company News Agent
 model: projects/990868019953/locations/us-central1/publishers/google/models/gemini-1.5-pro-001
 instructions: You are an expert in company information and news. You have access to sources like articles, companies and the people working there to answer user questions.]

In [ ]:
# app.delete_agent('Company News Agent')

### Registering the Extension

We register our extension(s) with the Agent.
You can add multiple function declarations and extensions at once.

One thing to *note*: Besides the OpenAI spec it makes a big difference to provide instructions for the agent again on when to use which tools and endpoints, and also giving some more hints about input formats.

Normally I would have expected for this to be retrieved from the OpenAPI spec, but it makes it more reliable to add the instructions for the tools here again.

In [34]:
INSTRUCTIONS = """
You are an expert in company information and news.
You have access to sources like articles, companies and the people working there to answer user questions.
If a user asks about companies in an industry, find the correct industry name first from the list and the use that to gather the companies.
If a user asks about articles in a specific month provide the date as yyyy-mm-dd format.
Companies mentioned in articles can be retrieved by the article id.
You can get detail for a company by company id or deatils for an article with the article id.
Also the people working for a company can be retrieved via the company id.

If the user asks you to compute a numeric value or analyse data use the code interpreter.

Only respond based on the response of the tools. Do not create your own answers.
"""

updated_agent = created_agent.update(
    new_instructions=INSTRUCTIONS,
    new_model="projects/{}/locations/us-central1/publishers/google/models/gemini-1.5-pro".format(PROJECT_ID),
#    new_functions=[function_declaration],
    new_extensions={
#        'Code Interpreter': code_interpreter_extension,
        'Company News Extension': company_news_extension
    }
)

In [ ]:
updated_agent

## Agent Management

### Listing Agents

You can list all the agents exiting under an App. This will return operatable Agent objects.

In [36]:
app.list_agents()

[
 agent_name: projects/990868019953/locations/us-central1/apps/992199292905062400/agents/5448665605571870720
 display_name: Company News Agent
 model: projects/vertex-ai-neo4j-extension/locations/us-central1/publishers/google/models/gemini-1.5-pro
 instructions: 
 You are an expert in company information and news.
 You have access to sources like articles, companies and the people working there to answer user questions.
 If a user asks about companies in an industry, find the correct industry name first from the list and the use that to gather the companies.
 If a user asks about articles in a specific month provide the date as yyyy-mm-dd format.
 Companies mentioned in articles can be retrieved by the article id.
 You can get detail for a company by company id or deatils for an article with the article id.
 Also the people working for a company can be retrieved via the company id.
 
 If the user asks you to compute a numeric value or analyse data use the code interpreter.
 
 Only res

###Get Agent

You can also get a specific Agent using either the resource name or the display name

In [37]:
agent = app.get_agent('Company News Agent')

Alternatively, you can get an Agent using the Agent class

Note that the only way to create a new Agent is to create it under an App, using `app.add_agent()`. The Agents constructor can only be used to get a reference to an Agent that has already been created.

In [ ]:
# agent = Agent(agent_id='projects/<project id>/locations/<location id>/<app id>/agents/<agent id>')

### Delete Agent
Using either the display name or the fully qualified resource name, you can delete a specific agent under the App.

In [ ]:
# app.delete_agent('<Agent Name>')

#Let the Magic Start - User Conversation Session

I must say I've been really impressed by the following conversations and the ability of the LLM to use the tools correctly until all information to answer the question has been collected.

The LLM is able to not only use the endpoints once, but also refer back to information from the conversation history and ensure the relevant context is taken into account.

It can fix misspelled or incorrect parameters to their correct content, reformat dates in the required shape and more. API functions are called multiple times and also cascading if needed. For instance first retrieve a list of id's for a certain context and then make individual calls for each id to retrieve details.

It is also able to resolve pronouns and demonstratives, e.g. "these" is resolved to a concrete list of company id's or articles that are then used to make multiple API calls.

I'll comment on each question what the trickiness is and how the LLM agent resolved it.

NOTE: you can remove `.content` from the end of each run call to get more detailed information about each session action

In [67]:
session = app.start_session()

INFO:google.cloud.aiplatform.private_preview.vertex_agents.app:Session init
INFO:google.cloud.aiplatform.private_preview.vertex_agents.app:Session init 2
INFO:google.cloud.aiplatform.private_preview.vertex_agents.app:app name = projects/990868019953/locations/us-central1/apps/992199292905062400
INFO:google.cloud.aiplatform.private_preview.vertex_agents.app:Creating session without request


## Multi-turn conversations with tool calls as needed

### Single API call with limiting response

Initially we do an easy one, the task here is to call the industries endpoint but then only limit it to 5 results (the endpoint takes no limit argument)

> Give me 5 industry names

In [68]:
result = session.run("Give me 5 industry names")

In [69]:
result.content

role: "model"
parts {
  text: "The first 5 industries are: Electronic Products Manufacturers, Enterprise Software Companies, Computer Hardware Companies, Business Software Companies, Mobile Phone Manufacturers. \n"
}

The result object has a lot of details, including the API calls made, the responses and possible reasoning of the model. We have `actions` which can be `user` or model `messages` or `tool_uses` calls and their responses.

The `content` attribute holds the final response with `role`and `parts`.

We get see the history of the seesion at any time, either to show it to the user or for debugging.

In [ ]:
session.get_history()

### Using prior information, fixing parameters and post-filtering

Here in this question, we provide a wrong spelling of the industry `Computer Hardware` instead of `Computer Hardware Companies` and an additional location that is not a parameter but needs to be post filtered.

> What are 3 companies in the 'Computer Hardware' space in California and their ids?

It uses the prior information in the conversation to fix the industry name and applies the post filter and limit to only show 3 Californian companies.

In [70]:
result = session.run("What are 3 companies in the 'Computer Hardware' space in California and their ids?")

In [71]:
print(result.content.parts[0].text)

Here are 3 companies in the 'Computer Hardware' space located in California: Apigee (EAHSCRG21ODCH1xk3354mxA), Xactly (E3lZVAk_KOpeQsRX4AS6F8w), Aerohive Networks (Ec2-e0bEXPuOQvE1KyUfLJg). 



### Reference back and requiring multiple tool invocations

In our next question we reference back with a `these` for the companies, and ask for details that are available in a separate endpoint. But the agent has to call the `people_at_company` endpoint multiple times, as it only takes a single company id.

> What people are working at these companies in which roles?


In [72]:
result = session.run("What people are working at these companies in which roles?")

In [73]:
print(result.content.parts[0].text)

At Apigee (EAHSCRG21ODCH1xk3354mxA), Promod Haque, Bob L Corey, and Neal Dempsey are on the Board of Directors. At Xactly (E3lZVAk_KOpeQsRX4AS6F8w), Chris Cabrera (CEO), Jeff Wilson, Brian Sheth, Betty Hung, and Dave Pidwell are on the Board. At Aerohive Networks (Ec2-e0bEXPuOQvE1KyUfLJg), David K Flynn is the CEO. 



### Parameter conversion, result sorting

Here we want to do two things, switching the conversation from companies to articles without restarting the session.

We give a partial date `January 2021`, that has to be converted to the format `yyyy-mm-dd` for the API call.

It has to understand that "good sentiment" refers to the numeric score and that we want to have the highest score.  The list of article that come with sentiment output, the model needs to read, understand, sort by sentiment descending

And finally select the 3 highest ranked articles and output their titles.

> What 3 articles in January 2021 had a really good sentiment?

In [74]:
result = session.run("What 3 articles in January 2021 had a really good sentiment?")

In [75]:
print(result.content.parts[0].text)

These articles from January 2021 had a sentiment score of 0.8 or higher: Accenture Acquires Wolox, Boosting Cloud First and Digital Transformation Capabilities in Argentina and South America (ART41721547195, sentiment: 0.892), Ubergizmo’s Best of CES 2021 (ART166350129247, sentiment: 0.912), Navisite Snaps Up Velocity Technology Solutions (ART269940336617, sentiment: 0.895). 



### Multiple retrieval calls plus summarization

Again the agent is asked to refer to the articles from the conversation before and get their ids.
Then it needs to make 3 calls to the `article` details endpoint to get the actual text.
And then use the LLM language skills to generate the summary.

> Can you summarize the text content of these articles?

In [76]:
result = session.run("Can you summarize the text content of these articles?")
result.content

role: "model"
parts {
  text: "Accenture Acquires Wolox, Boosting Cloud First and Digital Transformation Capabilities in Argentina and South America: This article reports that Accenture acquired Wolox, an Argentinian cloud native and agile development company. The acquisition will boost Accenture\'s cloud and digital transformation capabilities in Argentina and South America.\n\nUbergizmo’s Best of CES 2021: This article presents Ubergizmo\'s picks for the best products from CES 2021, including the Samsung Galaxy S21 series, Acer Predator Triton 300 SE, ASUS ROG Flow X13, Qualcomm 3D Sonic Sensor Gen 2, and more. \n\nNavisite Snaps Up Velocity Technology Solutions: The article discusses Navisite\'s acquisition of Velocity Technology Solutions, a cloud managed service provider. The acquisition aims to enhance Navisite\'s position in managed cloud and ERP services, expanding its offerings for mid-market and enterprise customers. \n"
}

### Refer back to a before previous information and disambiguate

Here we point back to the articles from the before previous question and ask for the companies mentioned. Again multiple API calls plus a de-duplication step at the end.

> Which companies were mentioned by these articles?

In [77]:
result = session.run("Which 3 companies were mentioned by these articles?")
print(result.content.parts[0].text)

The articles mentioned these 3 companies: Accenture (E6FeeEA51OSGCZak-XnjwpA), Wolox (E4UUIpJLLPAi-cVy1f2SmhQ), and Navisite (EV7uONsHPMnixTJcPkc3jKQ). 



### Vague reference resolution and multiple API calls

Here we refer back to the companies from before with "them" and also specify the roles requirement for the API call choice only vaguely.
The Agent needs to call the `people_at_company` endpoint multiple times, with the id's of the companies retrieved earlier in the session history and then aggregate the information into the response.

> And who are the people that work there in which roles?

In [78]:
result = session.run("And who are the people that work there in which roles?")
print(result.content.parts[0].text)

At Wolox (E4UUIpJLLPAi-cVy1f2SmhQ), Agustina Fainguersch is the CEO. At Navisite (EV7uONsHPMnixTJcPkc3jKQ), Larry W Schwartz, Andy Ruhan, Scott G. Pasquini, and Arthur Becker are on the board and Mark Clayman is the CEO. I do not have information about people working at Accenture. 



## Get conversation history

You can view the entire conversation history by using `session.get_history()`

In [ ]:
session.get_history()

Alternatively, you can list all the sessions your app has run and access or output the details in a structured way.

In [ ]:
sessions = app.list_sessions()

In [ ]:
print(sessions[0])

## Cleanup

Each abstraction has means to remove / clean up at runtime (sessions) or for the deployment (agent, application)

* delete session (optionally)
* delete agent (optionally)
* delete application

In [ ]:
# app.delete_session('<Session Name>')

In [ ]:
# App.delete(app_name="projects/990868019953/locations/us-central1/apps/3266798579703873536")

## Hosting and Deployment

To host our application a simple front-end like streamlit or gradio can be used, or it can be packaged into an [API with Reasoning Engine](https://www.googlecloudcommunity.com/gc/Cloud-Product-Articles/GenAI-GraphRAG-and-AI-agents-using-Vertex-AI-Reasoning-Engine/ta-p/789066) or Cloud Run.

# Conclusion

We demonstrated creating an agentic GenAI application that uses an external tool with a number of endpoints backed by a Neo4j Companies Graph. The appliciation is using the Vertex AI Agent SDK to make those endpoints available for information retrieval and interaction to the LLM for the human conversation.

We took the following steps

1. Setup the Neo4j Database
2. Build and Deploy the REST API Endpoints
3. Describe the REST API with an OpenAPI Specification
4. Create the application and register the endpoint extension to an agent
5. Demonstrate the application in a complex conversation that requires combination of memory usage and chained endpoint calls


It was really easy to integrate the endpoints that connect to the Neo4j graph database with the Agent SDK to the GenAI application. The main effort was adding detailed descriptions and instructions the API spec and registration.

The conversational capabilities were really impressive. Even for indirect and vague questions, the agent, using Gemini 1.5 Pro, was able to retrieve information either from the conversation history and previous responses or do multiple and even chained tool calls to get to the level of detailed needed to provide the answer.

Try it yourself. You find more information about the [Vertex AI Agent SDK](https://cloud.google.com/vertex-ai/generative-ai/docs/agent-api/overview) and the [GenAI capabilities of the Neo4j Graph Database](https://neo4j.com/generativeai).
